# Deep Demand Forecasting with Amazon SageMaker

In [ ]:
%%bash
python -m pip install --upgrade -q pip
python -m pip install -q gluonts

In [ ]:
import sagemaker

session = sagemaker.Session()
role = sagemaker.get_execution_role()

## Copy raw data to S3

The dataset we use here is the multivariate time-series [electricity consumptions](https://archive.ics.uci.edu/ml/datasets/ElectricityLoadDiagrams20112014) data taken from Dua, D. and Graff, C. (2019). [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php), Irvine, CA: University of California, School of Information and Computer Science. A cleaned version of the data containing **321** time-series with **1 Hour** frequency, starting from **2012-01-01** with **26304** time-steps, is available to download directly via [GluonTS](https://gluon-ts.mxnet.io/). We have also provided the [exchange rate](https://github.com/laiguokun/multivariate-time-series-data/tree/master/exchange_rate) dataset in case you want to try with other datasets as well.

For the ease of access, with have made both of the cleaned datasets available in the following S3 bucket

In [ ]:
DATASET_NAME = 'exchange_rate'
assert DATASET_NAME in ['electricity', 'exchange_rate']
NUM_TS = 321 if DATASET_NAME == 'electricity' else 8

In [ ]:
from sagemaker.s3 import S3Downloader

original_data_bucket = 'sagemaker-solutions-us-west-2'
original_data_prefix = 'sagemaker-deep-demand-forecast/{}'.format(DATASET_NAME)
original_data = 's3://{}/{}'.format(original_data_bucket, original_data_prefix)
print("original data: ")
S3Downloader.list(original_data)

First, setup the S3 bucket name and prefix

In [ ]:
bucket = session.default_bucket()  # default or use the name when creating the stack
prefix = 'tst'  # example
raw_data = 's3://{}/{}'.format(bucket, prefix)

Copy the `original_data` to our `raw_data` if does not exist already

In [ ]:
import boto3

if not S3Downloader.list(raw_data):
    s3 = boto3.client('s3')
    for file in s3.list_objects(Bucket=original_data_bucket, Prefix=original_data_prefix)['Contents']:
        copy_source = {
          'Bucket': original_data_bucket,
          'Key': file['Key']
        }
        s3.copy(copy_source, bucket, file['Key'].replace(original_data_prefix, prefix))

Set a few variables that will be used throughout the notebook

In [ ]:
preprocessed_data = 's3://{}/{}/processed_data'.format(bucket, prefix)
train_data = preprocessed_data
train_output = 's3://{}/{}/output'.format(bucket, prefix)
code_location = 's3://{}/{}/code'.format(bucket, prefix)

## Build container for Preprocessing and Feature Engineering

Data preprocessing and feature engineering is an important component of the ML lifecycle, and Amazon SageMaker Processing allows you to do these easily on a managed infrastructure. Here, we create a lightweight container that will serve as the environment for our data preprocessing. The container can also be easily customized to add more dependencies when needed.

In [ ]:
import boto3

region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'sagemaker-deep-demand-forecast-preprocessing-container'
ecr_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account_id,
                                                                    region,
                                                                    ecr_repository)

!bash preprocess/container/build_and_push.sh $ecr_repository docker

### Run Preprocessing job with Amazon SageMaker Processing

The script `src/preprocess/preprocess.py` max-normalizes the training data (correctly) and uses the found scales to normalize the testing data. We use SageMaker `ScriptProcessor` to perform these transformations on the `raw_data`.

In [ ]:
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(command=['python3'],
                                   image_uri=ecr_repository_uri,  # build and push
                                   role=role,
                                   instance_count=1,
                                   instance_type='ml.c4.xlarge')

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor.run(code='preprocess/preprocess.py',
                     inputs=[ProcessingInput(source=raw_data,
                                             destination='/opt/ml/processing/input')],
                     outputs=[ProcessingOutput(destination=preprocessed_data,
                                                source='/opt/ml/processing/output')],
                     arguments=[f'--dataset-name={DATASET_NAME}'],
                     logs=True,
                    )

### View Results of Data Preprocessing

Once the preprocessing job is complete, we can take a look at the contents of the S3 bucket.

In [ ]:
from sagemaker.s3 import S3Downloader
processed_files = S3Downloader.list(preprocessed_data)
print('\n'.join(processed_files))
S3Downloader.download(preprocessed_data, preprocessed_data.split("/")[-1])

## Train your LSTNet model with GluonTS

**LSTNet** is a Deep Learning model that incorporates traditional *auto-regressive* linear models *in parallel* to the non-linear neural network part, which makes the *non-linear* deep learning model more *robust* for the time series which *violate scale changes*. 

For more details, please checkout the paper [Modeling Long- and Short-Term Temporal Patterns with Deep Neural Networks](https://arxiv.org/abs/1703.07015).

### Hyperparameters

Here is a set of hyperparameters for LSTNet model for train for **5 epoch**

In [ ]:
CONTEXT_LENGTH = 12
PREDICTION_LENGTH = 6

hyperparameters = {
    'context_length': CONTEXT_LENGTH,
    'prediction_length': PREDICTION_LENGTH,
    'skip_size': 4,
    'ar_window': 4,
    'channels': 72,
    'scaling': False,
    'output_activation': 'sigmoid',  # either None, sigmoid or tanh
    'epochs': 5,
    'batch_size': 32,
    'learning_rate': 1e-2,
}

### Create and Fit SageMaker Estimator

With the hyperparameters defined, we can execute the training job. We will be using the [GluonTS](https://gluon-ts.mxnet.io/), with **MXNet** as the backend deep learning framework, to define and train our *LSTNet* model. **Amazon SageMaker** makes it do this with the Framework estimators which have the deep learning frameworks already setup. Here, we create a SageMaker MXNet estimator and pass in our model training script, hyperparameters, as well as the number and type of training instances we want.

We can then `fit` the estimator on the the training data location in S3.

In [ ]:
import logging
from sagemaker.mxnet import MXNet

estimator = MXNet(entry_point='train.py',
                  source_dir='deep_demand_forecast',
                  role=role,
                  train_instance_count=1,
                  train_instance_type='ml.p3.2xlarge', # or 'ml.c4.2xlarge' without GPU
                  framework_version="1.6.0",
                  py_version='py3',
                  hyperparameters=hyperparameters,
                  output_path=train_output,
                  code_location=code_location,
                  sagemaker_session=session,
                  # container_log_level=logging.DEBUG,  # display debug logs
                 )

estimator.fit(train_data)

### Examine the training evaluation

We can now access the training artifacts from the specified `output_path` in the above estimator and visual the training results

In [ ]:
output_files = S3Downloader.list(train_output)
print('\n'.join(output_files))

In [ ]:
import os
output_path = os.path.join(train_output, estimator._current_job_name, 'output')

S3Downloader.download(output_path, 'output')
!tar -xvf output/output.tar.gz -C output/

In [ ]:
import pandas as pd

item_metrics = pd.read_csv('output/item_metrics.csv.gz', compression='gzip')
item_metrics.head()

### Visualizing the outputs

For the visualization we will use [altair package](https://github.com/altair-viz/altair) with declarative API. If you want to export to different file formats, follow [altair_saver](https://github.com/altair-viz/altair_saver). 

Note that after exporting to `html` you can go to `output` and open the generated `html` files inside notebook.

Here, we compare the [**Mean Absolute Scaled Error (MASE)**](https://en.wikipedia.org/wiki/Mean_absolute_scaled_error) against the [**symmetric Mean Absolute Percentage Error (sMAPE)**](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error)

In [ ]:
!python -m pip install -q altair==4.1

In [ ]:
import altair as alt

col_a = 'MASE'
col_b = 'sMAPE'

scatter = alt.Chart(item_metrics).mark_circle(size=100, fillOpacity=0.8).encode(
    alt.X(col_a, scale=alt.Scale(domain=[-0.5, 10])),
    alt.Y(col_b, scale=alt.Scale(domain=[0, 2.5])),
    tooltip=[col_a, col_b]
).interactive()
scatter.save(os.path.join('output', f'{col_a}_vs_{col_b}.html'))
scatter

In [ ]:
col_a_plot = alt.Chart(item_metrics).mark_bar().encode(
        alt.X(col_a, bin=True),
        y='count()',
)
col_b_plot = alt.Chart(item_metrics).mark_bar().encode(
    alt.X(col_b, bin=True),
    y='count()',
)

col_a_b_plot = col_a_plot | col_b_plot
col_a_b_plot.save(os.path.join('output', f'{col_a}_{col_b}_barplots.html'))
col_a_b_plot

## Deploy an endpoint

To serve the model, we can deploy an endpoint where the `src/deep_demand_forecast/inference.py` script handles the predictions using the trained model as follows

In [ ]:
from sagemaker.mxnet import MXNetModel

model = MXNetModel(model_data=os.path.join(output_path, 'model.tar.gz'),
                   role=role,
                   entry_point='inference.py',
                   source_dir='deep_demand_forecast',
                   py_version='py3',
                   framework_version='1.6.0',
                  )

predictor = model.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1)

### Testing the endpoint

To do sanity checking, here we can test the endpoint by requesting predictions for a randomly generated data. The `predictor` handles serialization and deserialization of the requests.

In [ ]:
import numpy as np

np.random.seed(1)
random_test = np.random.randn(NUM_TS, PREDICTION_LENGTH)

# json serializable request format
random_test_data = {}
random_test_data['target'] = random_test.tolist()
random_test_data['start'] = '2014-01-01'
random_test_data['source'] = []

random_ret = predictor.predict(random_test_data)

and then loads the return JSON objects

In [ ]:
import json

forecasts = np.array(random_ret['forecasts']['samples'])
print('Forecasts shape with 10 samples: {}'.format(forecasts.shape))
print('RMSE: {}'.format(json.loads(random_ret['agg_metrics'])['RMSE']))

## Interactive visualization

It is important to visualize how the model is performing given the test data. Since we have trained our model given the hyperparameters defined earlier `CONTEXT_LENGTH` (input length) and `PREDICTION_LENGTH` (output length), we can now input the final window to our model where the last training time is **2014-05-26 19:00:00** so we test from **2014-05-26 19:00:00** onwards and get the predictions and visualize the perfomance of the model for a sample of time-series.

We have provided some utilities in `deep_demand_forecast/monitor.py` for creating the visualization data from train, test and predictions.

In [ ]:
%run deep_demand_forecast/monitor.py

In [ ]:
train_df, test_df = prepare_data('processed_data')
print(f'prepared train data shape {train_df.shape}, test data shape {test_df.shape}')
# print(f"train data head {train_df.head()}")
ts_col_names = [f'ts_{i}' for i in range(NUM_TS + 1)]
train_df_viz, test_df_viz, selected_cols = create_data_viz(train_df, test_df, CONTEXT_LENGTH, PREDICTION_LENGTH, num_sample_ts=11)
train_df_viz.head()

In [ ]:
selection = alt.selection_multi(fields=['covariate'], bind='legend', nearest=True)

train_plot = alt.Chart(train_df_viz, title='Train data').mark_line().encode(
    alt.X('time:T', axis=alt.Axis(title='Time')),
    alt.Y('value:Q', axis=alt.Axis(title='Normalized electricity consumption (kW)')),
    alt.Color('covariate:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).add_selection(
    selection
)

test_plot = alt.Chart(test_df_viz, title='Test data').mark_line().encode(
    alt.X('time:T', axis=alt.Axis(title='Time')),
    alt.Y('value:Q', axis=alt.Axis(title='Normalized electricity consumption (kW)')),
    alt.Color('covariate:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).add_selection(
    selection
)

train_plot | test_plot

Here we prepare our test data for prediction as before

In [ ]:
num_train = train_df.shape[0] - 1
test_data = {}
test_data['target'] = test_df.iloc[num_train: num_train + PREDICTION_LENGTH].set_index('time').values.T.tolist()
test_data['start'] =  '2014-05-26 19:00:00'
test_data['source'] = []

predictions = predictor.predict(test_data)

And finally prepare for the final interactive visualization

In [ ]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

forecasts = np.transpose(np.array(predictions['forecasts']['samples'][0]))
preds = ListDataset([{FieldName.TARGET: forecasts,
                           FieldName.START: predictions['forecasts']['start_date']
                      }], freq=predictions['forecasts']['freq'], one_dim_target=False)

preds_df = multivar_df(next(iter(preds)))
preds_df_filter = preds_df.loc[:, ['time'] + selected_cols]
preds_df_filter = pd.melt(preds_df_filter, id_vars=['time'], value_vars=selected_cols)
preds_df_filter.rename(columns={'variable': 'covariate'}, inplace=True)
preds_df_filter.head()

In [ ]:
preds_plot = alt.Chart(preds_df_filter, title='Predictions').mark_line().encode(
    alt.X('time:T', axis=alt.Axis(title='Time')),
    alt.Y('value:Q', axis=alt.Axis(title='Normalized electricity consumption (kW)')),
    alt.Color('covariate:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).add_selection(
    selection
)

(train_plot | test_plot) & preds_plot

## Optional: Delete the endpoint and model

When you're done with the endpoint, you should clean it up.

All of the training jobs, models and endpoints we created can be viewed through the SageMaker console of your AWS account.

In [ ]:
predictor.delete_endpoint()

In [ ]:
predictor.delete_model()